In [5]:
1/8

0.125

In [115]:
import cProfile
import time

import numpy as np
import matplotlib.pyplot as plt
from numpy.random import choice

pe = 0.8
pm = 0.8
pn = 0.2
pi = 0.2
pc = 0.8
nb_repeats = 1000
nb_steps = 600
seed = np.random.randint(10000)
print(seed)

#%%time

def vectorized():
    np.random.seed(seed)
    markovburn = 500
    """
    # trans_prob initialise (neutral)
    trans_prob = {
        0: [1-pe, pe],  # Probabilities for transitioning from E=0
        1: [1-pe, pe],  # Probabilities for transitioning from E=1
    }
    """
    trans_prob = {
            0: [0.9, 0.1],  # Probabilities for transitioning from E=0
            1: [0.4, 0.6],  # Probabilities for transitioning from E=1
        }
    
    #print(f'seed in etrans {random_seed}')
    #print(f'pe {pe}')
    #print(f'trans_prob in etrans {trans_prob[0]}')

    # Initialize numpy arrays for M, A, and V
    arr_M = np.zeros(nb_repeats, dtype=int)
    #arr_E = np.zeros(nb_repeats, dtype=int)
    arr_A = np.zeros((nb_repeats, nb_steps - markovburn), dtype=int) # Discard first 500 items to allow Markov chain burn-in
    arr_V = np.zeros((nb_repeats, nb_steps - markovburn), dtype=int)

    # Vectorize initial M and first value of E for all trials
    arr_M = choice([-1, 0, 1], size=nb_repeats, p=[pm / 2, 1 - pm, pm / 2])
    E_initial = choice([0, 1], size=nb_repeats, p=[1 - pe, pe])

    # Initialize E for all trials and steps
    E_all_trials = np.zeros((nb_repeats, nb_steps), dtype=int)
    E_all_trials[:, 0] = E_initial



    for trial in range(nb_repeats):
        #if run == 0: # Test run
        #    M = 1
        #else:
        #M = choice([-1, 0, 1], p=[pm / 2, 1 - pm, pm / 2])
        #arr_M[trial] = M
        E = E_all_trials[trial]
        M = arr_M[trial]
        #E = np.zeros(nb_steps, dtype=np.int32)
        if M:
            #E[0] = choice([0, 1], p=[1 - pe, pe]) # First value of E
            for i in range(1, nb_steps):  # Select one value in E at a time
                E[i] = choice([0, 1], p=trans_prob[E[i - 1]])

        # Discard initial segment of E to obtain Markov chain that has attained equilubrium
        E = E[markovburn:]

        vals_no_e = np.array([-1, 0, 1]) # Add noise if E = 0
        p_no_e = np.array([pn / 2, 1 - pn, pn / 2])

        vals_e = np.array([-M, 0, M])
        p_e = np.array([pi, 1 + (- pc - pi), pc])

        #A = np.where(E, choice(vals_e, size=nb_steps, p=p_e), choice(vals_no_e, size=nb_steps, p=p_no_e))
        #V = np.where(E, choice(vals_e, size=nb_steps, p=p_e), choice(vals_no_e, size=nb_steps, p=p_no_e))
        
        A = np.where(E, choice(vals_e, size=E.size, p=p_e), choice(vals_no_e, size=E.size, p=p_no_e))
        V = np.where(E, choice(vals_e, size=E.size, p=p_e), choice(vals_no_e, size=E.size, p=p_no_e))
        

        arr_A[trial, :] = A 
        arr_V[trial, :] = V
    return arr_A, arr_V, E, arr_M
        

#%%time

def current():    
    np.random.seed(seed)
    markovburn = 500
    """
    # trans_prob initialise (neutral)
    trans_prob = {
        0: [1-pe, pe],  # Probabilities for transitioning from E=0
        1: [1-pe, pe],  # Probabilities for transitioning from E=1
    }
    """
    trans_prob = {
            0: [0.9, 0.1],  # Probabilities for transitioning from E=0
            1: [0.4, 0.6],  # Probabilities for transitioning from E=1
        }
    #print(f'seed in etrans {random_seed}')
    #print(f'pe {pe}')
    #print(f'trans_prob in etrans {trans_prob[0]}')

    # Initialize numpy arrays for M, A, and V
    arr_M = np.zeros(nb_repeats, dtype=int)
    #arr_E = np.zeros(nb_repeats, dtype=int)
    arr_A = np.zeros((nb_repeats, nb_steps - markovburn), dtype=int) # Discard first 500 items to allow Markov chain burn-in
    arr_V = np.zeros((nb_repeats, nb_steps - markovburn), dtype=int)
    for trial in range(nb_repeats):
        #if run == 0: # Test run
        #    M = 1
        #else:
        M = choice([-1, 0, 1], p=[pm / 2, 1 - pm, pm / 2])
        arr_M[trial] = M
        E = np.zeros(nb_steps, dtype=np.int32)
        if M:
            E[0] = choice([0, 1], p=[1 - pe, pe]) # First value of E
            for i in range(1, nb_steps):  # Select one value in E at a time
                E[i] = choice([0, 1], p=trans_prob[E[i - 1]])

        # Discard initial segment of E to obtain Markov chain that has attained equilubrium
        E = E[markovburn:]

        vals_no_e = np.array([-1, 0, 1]) # Add noise if E = 0
        p_no_e = np.array([pn / 2, 1 - pn, pn / 2])

        vals_e = np.array([-M, 0, M])
        p_e = np.array([pi, 1 + (- pc - pi), pc])

        #A = np.where(E, choice(vals_e, size=nb_steps, p=p_e), choice(vals_no_e, size=nb_steps, p=p_no_e))
        #V = np.where(E, choice(vals_e, size=nb_steps, p=p_e), choice(vals_no_e, size=nb_steps, p=p_no_e))
        
        A = np.where(E, choice(vals_e, size=E.size, p=p_e), choice(vals_no_e, size=E.size, p=p_no_e))
        V = np.where(E, choice(vals_e, size=E.size, p=p_e), choice(vals_no_e, size=E.size, p=p_no_e))
        

        arr_A[trial, :] = A 
        arr_V[trial, :] = V
    
    return arr_A, arr_V, E, arr_M


4043


In [118]:
%%timeit
arr_A, arr_V, E_all_trials, arr_M = current()


6.73 s ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [120]:
%%timeit
arr_A_vec, arr_V_vec, E_all_trials_vec, arr_M_vec = vectorized() 


5.89 s ± 45.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def stats():
    print('E    ', np.bincount(E_all_trials))  
    print('E vec', np.bincount(E_all_trials_vec))
    print('M    ', np.bincount(arr_M + 1))  
    print('M vec', np.bincount(arr_M_vec + 1))    
stats()